In [1]:
import dijkstra3d
import numpy as np
import os
import pandas as pd
import SimpleITK as sitk
from skimage import data
import Functions as F 
import matplotlib.pyplot as plt 

In [19]:
def read_from_df(df, idx, nr, which): 
    position = np.array([df.loc[nr, which + " " + str(idx) + " pos x"], df.loc[nr, which + " " + str(idx) + " pos y"], df.loc[nr, which + " " + str(idx) + " pos z"],])
    return position


def run_dijkstra(results, nr_agents, which, con_chorda, con_facial):
    ind = 0
    data = pd.read_csv(results)
    weights = np.zeros((data.shape[0], 2))
    idx = F.get_best_agents(data) if nr_agents == 12 else [0, 1, 2, 3, 4, 5]
    with open(r"Cmarl\src\data\filenames\testing.txt", 'r') as file: 
        filenames = file.read().split("\n")


    for nr in range(data.shape[0]):
        if nr >= 9: 
            id = 41
        else: 
            id = 36
        print(f"nr = {nr}")
        C = read_from_df(data, idx[0], nr, which)
        A = read_from_df(data, idx[1], nr, which)
        R = read_from_df(data, idx[2], nr, which)
        M = read_from_df(data, idx[3], nr, which)
        T = read_from_df(data, idx[4], nr, which)

        #load image
        mr_scan = sitk.GetArrayFromImage(sitk.ReadImage(filenames[nr]))

        #positive (but mr is always positive)
        field = mr_scan
                
        # path is an [N,3] numpy array i.e. a list of x,y,z coordinates
        # terminates early, default is 26 connected
        # conectivity 18 2 landmarks
        path_chorda = dijkstra3d.dijkstra(field, A, C, connectivity=con_chorda, bidirectional=True)
        
        label = mr_scan.copy()*0
        for (x,y,z) in path_chorda:
            label[x,y,z] = 1
        sitk.WriteImage(sitk.GetImageFromArray(label), os.path.join('paths/paths_chorda',filenames[nr][id:])  , useCompression=True)
        np.savetxt("paths/chordatxt/" + filenames[nr][id:-6] + "txt", path_chorda, fmt='%s')
        
        
        #here we use 3 landmarks so there are two paths
        path_facialRM = dijkstra3d.dijkstra(field, R, M, connectivity = con_facial, bidirectional=True)

        label_26 = mr_scan.copy()*0
        for (x,y,z) in path_facialRM:
            label_26[x,y,z] = 1

        np.savetxt("paths/facialtxtRM/" + filenames[nr][id:-6] + "txt", path_facialRM,fmt='%s', delimiter = ",")

        path_facialMT = dijkstra3d.dijkstra(field, M, T, connectivity = con_facial, bidirectional=True)
        for (x,y,z) in path_facialMT:
            label_26[x,y,z] = 1

        sitk.WriteImage(sitk.GetImageFromArray(label_26), os.path.join('paths/paths_facial',filenames[nr][id:])  , useCompression=True)
        np.savetxt("paths/facialtxtMT/" + filenames[nr][id:-6] + "txt", path_facialMT,fmt='%s', delimiter = ",")



        # Compute vector with intensisties for each candidate path

        val_chorda=[]
        for (x,y,z) in path_chorda:
            val_chorda.append(field[x,y,z])       
        
        val_facial = []
        for (x,y,z) in path_facialRM:
            val_facial.append(field[x,y,z])
        
        for (x, y, z) in path_facialMT:
            val_facial.append(field[x,y,z])


            
        val_chorda = np.array(val_chorda)
        val_facial = np.array(val_facial)
        
        # Compute comulative squared derivative
        rat_chorda = (np.sum(np.square(np.array(val_chorda[1:]-val_chorda[:-1], dtype='int64')))+np.sum(np.square(np.array(val_chorda[1:2:]-val_chorda[0:2:-1], dtype='int64'))))#/(len(val_26))
        rat_facial = (np.sum(np.square(np.array(val_facial[1:]-val_facial[:-1], dtype='int64')))+np.sum(np.square(np.array(val_facial[1:2:]-val_facial[0:2:-1], dtype='int64'))))#/(len(val_18))
        

        weights[ind,:]= [rat_chorda,rat_facial]
        ind += 1
    
    return weights 
    
    

In [20]:
results = "results/results03.csv"
nr_agents = 6
which = "Landmark"
con_chorda = 6
con_facial = 26  
weights = run_dijkstra(results, nr_agents, which, con_chorda, con_facial)

nr = 0
nr = 1
nr = 2
nr = 3
nr = 4
nr = 5
nr = 6
nr = 7
nr = 8
nr = 9
nr = 10
nr = 11
nr = 12
nr = 13
nr = 14
nr = 15
nr = 16
nr = 17
nr = 18
nr = 19
nr = 20
nr = 21


In [18]:
    with open(r"Cmarl\src\data\filenames\testing.txt", 'r') as file: 
        filenames = file.read().split("\n")
    
    filenames[9][41:]

'011_1_R.nii.gz'

In [66]:
Viz = F.VisualizeAngle("results/results03.csv", 6)
Viz.get_coor()
Viz.plot_angle_in_plane(10)
angles = Viz.compute_all_angles()
Rot = Viz.R

c:\Users\There\OneDrive\Dokumenter\GitHub\Bachelor-thesis-ear\Functions\angleVis.py:92: RuntimeWarning: invalid value encountered in true_divide
  n /= np.linalg.norm(n)


In [67]:
path_chorda = (Rot @ path_18.T).T 
path_ = np.concatenate([path_26[0], path_26[1]], axis = 0)

path_facial = (Rot @ path_.T).T 
xn = np.linspace(-100, 100)
m, c = np.polyfit(path_chorda[:, 0], path_chorda[:, 1], 1)
yn = np.polyval([m, c], xn)

m1, c1, = np.polyfit(path_facial[:, 0], path_facial[:, 1], 1)
yn2 = np.polyval([m1, c1], xn)


In [68]:
p1 = np.array([xn[0], yn[0]])
p2 = np.array([xn[-1], yn[-1]])
chorda_direction = p2 - p1 

p3 = np.array([xn[0], yn2[0]])
p4 = np.array([xn[-1], yn2[-1]])
facial_direction = p4 - p3

angle = np.arccos(chorda_direction @ facial_direction /(np.linalg.norm(chorda_direction) * np.linalg.norm(facial_direction))) * 180 / np.pi 

In [69]:
%matplotlib qt 
import matplotlib
matplotlib.rcParams.update({'font.size': 18})
plt.scatter(path_chorda[:, 0], path_chorda[:, 1], color = "b", label = "CTY") # A
plt.scatter(path_facial[:, 0], path_facial[:, 1], color = "r", label = "FN")
plt.axis("square")
plt.plot(xn, yn)
plt.plot(xn, yn2)
plt.legend()
plt.title("Angle = " + str(angle))
plt.xticks([])
plt.yticks([])
plt.show()

In [113]:
mu = np.mean(path_, axis = 0)
U, Sigma, V = np.linalg.svd(path - mu)
direction = V[0]

array([ 0.06068246, -0.15829528, -0.98552536])

In [63]:
for i in range(len(path_facial)): 
    print(path_facial[i, 1])

33.80766007663637
34.83399241849138
35.80757322223459
36.8586263959745
37.88495873782951
38.85522966723072
39.853531302858826
40.82711210660204
41.79738303600325
42.76765396540445
43.73792489480566
43.73792489480566
43.76595560103256
44.73953640477578
45.71311720851899
46.7114188441471
47.70972047977521
48.73274294728821
49.70301387668942
49.65026233857762
